# Inference of the U2OS interactome

---
In this notebook we will derive the interactome of U2OS cells using the Prize-collecting Steiner tree algorithm.

---

## 0. Environmental setup

In [1]:
import numpy as np
import pandas as pd
from pcst_fast import pcst_fast
import networkx as nx
import random
from tqdm import tqdm
import community

seed = 1234
np.random.seed(seed)
random.seed(seed)

In [2]:
def run_pcst_sensitivity_analyses(graph, bs, prize_key:str="prize", weight_key:str="cost"):
    node_dict = dict(zip(list(graph.nodes()), list(range(len(graph.nodes())))))
    inv_edge_dict = dict(zip(list(range(len(graph.edges()))), list(graph.edges())))
    
    vertices = list(node_dict.values())
    edges = []
    prizes = []
    costs = []
    for node in graph.nodes(data=True):
        prizes.append(node[-1][prize_key])
    for edge in graph.edges(data=True):
        edges.append((node_dict[edge[0]], node_dict[edge[1]]))
        costs.append(edge[-1][weight_key])
    
    edges = np.array(edges)
    prizes = np.array(prizes)
    costs = np.array(costs)
    
    pcs_tree_dict = {}
    augmented_pcs_tree_dict={}
    
    for b in tqdm(bs, desc="Compute PCSTs"):
        v_idc, e_idc = pcst_fast(edges, prizes*b, costs, -1, 1, "strong", 0)
        selected_edges = [inv_edge_dict[e_idx] for e_idx in e_idc]
        #print(selected_edges)
        pcs_tree = graph.edge_subgraph(selected_edges)
        augmented_pcs_tree = graph.subgraph(pcs_tree.nodes())
        pcst_name = graph.name + "_b_{}".format(b)
        pcs_tree_dict[pcst_name] = pcs_tree
        augmented_pcs_tree_dict[pcst_name+"_augmented"] = augmented_pcs_tree
    return pcs_tree_dict, augmented_pcs_tree_dict
    

In [3]:
def analyze_pcst_sensitivity_analyses_results(trees_dict, target_nodes):
    data = {
        "beta": [],
        "n_nodes": [],
        "n_edges": [],
        "n_connected_components": [],
        "n_louvain_clusters": [],
        "avg_node_degree": [],
        "std_node_degree": [],
        "n_leaf_nodes": [],
        "n_target_nodes": [],
        "n_target_leafs": [],
        "avg_target_degree": [],
        "std_target_degree": [],
    }
    keys = []
    for key, tree in tqdm(trees_dict.items(), desc="Analyze tree:"):
        keys.append(key)
        splitted = key.split("_")
        beta = splitted[-1]

        n_nodes = len(tree.nodes())
        n_edges = len(tree.edges())
        n_connected_components = nx.number_connected_components(tree)
        n_louvain_clusters = len(
            np.unique(list(community.best_partition(tree).values()))
        )
        node_degrees = []
        target_degrees = []
        leaf_nodes = []
        for node in tree.nodes():
            node_degree = tree.degree(node)
            node_degrees.append(node_degree)
            if node_degree == 1:
                leaf_nodes.append(node)
            if node in target_nodes:
                target_degrees.append(node_degree)
        avg_node_degree = np.mean(node_degrees)
        std_node_degree = np.std(node_degrees)
        avg_target_degree = np.mean(target_degrees)
        std_target_degree = np.std(target_degrees)
        n_target_leafs = len(set(target_nodes).intersection(set(leaf_nodes)))
        n_leaf_nodes = len(leaf_nodes)
        n_target_nodes = len(set(target_nodes).intersection(set(list(tree.nodes()))))

        data["beta"].append(float(beta))
        data["n_nodes"].append(n_nodes)
        data["n_edges"].append(n_edges)
        data["n_connected_components"].append(n_connected_components)
        data["n_louvain_clusters"].append(n_louvain_clusters)
        data["avg_node_degree"].append(avg_node_degree)
        data["std_node_degree"].append(std_node_degree)
        data["n_leaf_nodes"].append(n_leaf_nodes)
        data["n_target_nodes"].append(n_target_nodes)
        data["n_target_leafs"].append(n_target_leafs)
        data["avg_target_degree"].append(avg_target_degree)
        data["std_target_degree"].append(std_target_degree)

    data = pd.DataFrame.from_dict(data)
    data.index = keys
    return data

---

## 1. Read in data

In [4]:
## Version 1: Prizes = absolute logFC in CCLE data set, Weights = 1-|Spearman correlation|
ppi_v1 = nx.read_gpickle("../../data/ppi/ppi_confidence_pruned_0542_ccle_abslogfc_spearmanr.pkl")
ppi_v1.name = "Confidence_Pruned_CCLE_absLogFC_Spearman_r"
print(nx.info(ppi_v1))

## Version 1: Prizes = absolute logFC in CCLE data set, Weights = bootstrap p-value of non-zero Spearman correlation
ppi_v2 = nx.read_gpickle("../../data/ppi/ppi_confidence_pruned_0542_ccle_abslogfc_spearmanp.pkl")
ppi_v2.name = "Confidence_Pruned_CCLE_absLogFC_Spearman_p"
print(nx.info(ppi_v2))

## Version 3: Prizes = absolute logFC in CCLE data set or 10 if ORF target, Weights = 1-|Spearman correlation|
ppi_v3 = nx.read_gpickle("../../data/ppi/ppi_confidence_pruned_0542_ccle_abslogfc_orf_10_spearmanr.pkl")
ppi_v3.name = "Confidence_Pruned_CCLE_absLogFC_ORF_10_Spearman_r"
print(nx.info(ppi_v3))

## Version 4: Prizes = absolute logFC in CCLE data set or 10 if ORF target, Weights = bootstrap p-value of non-zero Spearman correlation
ppi_v4 = nx.read_gpickle("../../data/ppi/ppi_confidence_pruned_0542_ccle_abslogfc_orf_10_spearmanp.pkl")
ppi_v4.name = "Confidence_Pruned_CCLE_absLogFC__ORF_10_Spearman_p"
print(nx.info(ppi_v4))

Name: Confidence_Pruned_CCLE_absLogFC_Spearman_r
Type: Graph
Number of nodes: 10841
Number of edges: 54550
Average degree:  10.0636
Name: Confidence_Pruned_CCLE_absLogFC_Spearman_p
Type: Graph
Number of nodes: 10841
Number of edges: 54550
Average degree:  10.0636
Name: Confidence_Pruned_CCLE_absLogFC_ORF_10_Spearman_r
Type: Graph
Number of nodes: 10841
Number of edges: 54550
Average degree:  10.0636
Name: Confidence_Pruned_CCLE_absLogFC__ORF_10_Spearman_p
Type: Graph
Number of nodes: 10841
Number of edges: 54550
Average degree:  10.0636


In [5]:
orf_targets = set(
    pd.read_csv("../../data/other/selected_orf_targets.txt", index_col=0, header=None).index
)

---

## 2. Run PCST-based inference

Using the above data, we will approximately solve the PCST problem for different choice of $\beta$ in the PCST objectives, i.e. the prize factor. We aim to identify a stable subgraph from the confidence pruned humna PPI that describes the cell-type specific interactome of U2OS cells best.

### 2a. Interactome version 1

We will first infer the interactome for the first choice of input network, i.e. where the node prizes are given by the absolute logFC comparing U2OS cells against over 1300 other cancer cell lines in the CCLE data set and the edge weights are given by $1-|r_{Spearman}|$.

In [6]:
bs = np.arange(0,50,0.1)
pcst_dict_v1, augmented_pcst_dict_v1 = run_pcst_sensitivity_analyses(graph=ppi_v1, bs=bs)
pcst_results_v1 = analyze_pcst_sensitivity_analyses_results(pcst_dict_v1, target_nodes=orf_targets)

Analyze tree::   0%|          | 0/500 [00:00<?, ?it/s]/home/paysan_d/miniconda3/envs/image2reg/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/paysan_d/miniconda3/envs/image2reg/lib/python3.8/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/home/paysan_d/miniconda3/envs/image2reg/lib/python3.8/site-packages/numpy/core/_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/paysan_d/miniconda3/envs/image2reg/lib/python3.8/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/home/paysan_d/miniconda3/envs/image2reg/lib/python3.8/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value enc

### 2b. Interactome version 3

The only difference in the setup of version 3 is that we set the prize for all ORF targets to promote solutions that include those targets as we are interested in finding the interactome spanning those nodes and thus describing their regulatory relationships.

In [7]:
pcst_dict_v3, augmented_pcst_dict_v3 = run_pcst_sensitivity_analyses(graph=ppi_v3, bs=bs)
pcst_results_v3 = analyze_pcst_sensitivity_analyses_results(pcst_dict_v3, target_nodes=orf_targets)

Analyze tree:: 100%|██████████| 500/500 [11:40<00:00,  1.40s/it]


### 2c. Interactome version 2

The setup is as in version 1 but this time the edge weight is given by the bootstrap p-value for testing for zero Spearman correlation.

In [8]:
pcst_dict_v2, augmented_pcst_dict_v2 = run_pcst_sensitivity_analyses(graph=ppi_v2, bs=bs)
pcst_results_v2 = analyze_pcst_sensitivity_analyses_results(pcst_dict_v2, target_nodes=orf_targets)

Analyze tree:: 100%|██████████| 500/500 [12:22<00:00,  1.49s/it]


### 2d. Interactome version 4

As in version two but with ORF prizes set to 10.

In [9]:
pcst_dict_v4, augmented_pcst_dict_v4 = run_pcst_sensitivity_analyses(graph=ppi_v4, bs=bs)
pcst_results_v4 = analyze_pcst_sensitivity_analyses_results(pcst_dict_v4, target_nodes=orf_targets)

Analyze tree:: 100%|██████████| 500/500 [13:53<00:00,  1.67s/it]


In [12]:
pcst_results_v3

,beta,n_nodes,n_edges,n_connected_components,n_louvain_clusters,avg_node_degree,std_node_degree,n_leaf_nodes,n_target_nodes,n_target_leafs,avg_target_degree,std_target_degree
Confidence_Pruned_CCLE_absLogFC_ORF_10_Spearman_r_b_0.0,0.0,0,0,0,0,NaN,NaN,0,0,0,NaN,NaN
Confidence_Pruned_CCLE_absLogFC_ORF_10_Spearman_r_b_0.1,0.1,178,177,1,13,1.988764,1.565028,96,168,96,1.940476,1.576246
Confidence_Pruned_CCLE_absLogFC_ORF_10_Spearman_r_b_0.2,0.2,280,279,1,16,1.992857,1.449120,129,183,108,1.852459,1.509562
Confidence_Pruned_CCLE_absLogFC_ORF_10_Spearman_r_b_0.30000000000000004,0.3,441,440,1,20,1.995465,1.796943,220,185,108,1.924324,1.585305
Confidence_Pruned_CCLE_absLogFC_ORF_10_Spearman_r_b_0.4,0.4,733,732,1,26,1.997271,3.114897,408,185,97,2.124324,1.925842
...,...,...,...,...,...,...,...,...,...,...,...,...
Confidence_Pruned_CCLE_absLogFC_ORF_10_Spearman_r_b_49.5,49.5,10581,10580,1,159,1.999811,22.127689,7717,185,61,4.313514,5.673941
Confidence_Pruned_CCLE_absLogFC_ORF_10_Spearman_r_b_49.6,49.6,10582,10581,1,156,1.999811,22.126665,7718,185,61,4.313514,5.673941
Confidence_Pruned_CCLE_absLogFC_ORF_10_Spearman_r_b_49.7,49.7,10582,10581,1,154,1.999811,22.126665,7718,185,61,4.313514,5.673941
Confidence_Pruned_CCLE_absLogFC_ORF_10_Spearman_r_b_49.800000000000004,49.8,10582,10581,1,157,1.999811,22.126665,7718,185,61,4.313514,5.673941
